In [130]:
# import section

import keras
from keras import regularizers
from keras.models import load_model
from keras.models import  Model, Sequential
from keras.layers import Dense, Input, LSTM ,Embedding , TimeDistributed, BatchNormalization
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

## HDFS Time

In [131]:
struct_log = './data/HDFS_100k.log_structured.csv' # The structured log file
label_file = './data/HDFS_100k.log_anomaly_label.csv' # The anomaly label file

In [132]:
df = pd.read_csv(struct_log)
df.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating


In [133]:
from utils import dataloader
from utils.preprocessing import Vectorizer_sys


batch_size = 32
hidden_size = 32
num_directions = 2
topk = 5
train_ratio = 0.2
window_size = 10
epoches = 2
num_workers = 2
device = 0 

x_, window_y_ = \
dataloader.load_HDFS(struct_log, window='session',window_size=window_size, train_ratio=train_ratio, split_type='uniform', Time = True)

====== Input data summary ======
Loading ./data/HDFS_100k.log_structured.csv
Slicing 1406 sessions, with window 10
Slicing done, 96459 windows generated
Train: windows


In [134]:
feature_extractor = Vectorizer_sys()
train_dataset = feature_extractor.fit_transform(x_, window_y_)

In [135]:
train_dataset['x']

array([[4, 9, 1, ..., 1, 1, 1],
       [4, 4, 5, ..., 6, 3, 3],
       [4, 5, 5, ..., 3, 3, 3],
       ...,
       [0, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [136]:
train_dataset['x'] = train_dataset['x']/17

In [137]:
train_dataset['x']

array([[0.23529412, 0.52941176, 0.05882353, ..., 0.05882353, 0.05882353,
        0.05882353],
       [0.23529412, 0.23529412, 0.29411765, ..., 0.35294118, 0.17647059,
        0.17647059],
       [0.23529412, 0.29411765, 0.29411765, ..., 0.17647059, 0.17647059,
        0.17647059],
       ...,
       [0.        , 0.        , 0.05882353, ..., 0.05882353, 0.05882353,
        0.05882353],
       [0.        , 0.05882353, 0.05882353, ..., 0.05882353, 0.05882353,
        0.05882353],
       [0.        , 0.        , 0.        , ..., 0.05882353, 0.05882353,
        0.05882353]])

In [138]:
# Autoencoder model.
input_dim = train_dataset['x'].shape[1]
input = Input(shape=(input_dim, ))
encode = Dense(input_dim//3*2, activation='relu',kernel_regularizer=regularizers.l2(0.01))(input)
encode = BatchNormalization()(encode)
encode = Dense(input_dim//3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)
encode = Dense(1, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)

decode = Dense(input_dim//3, activation='relu')(encode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim//3*2, activation='relu')(decode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim, activation='sigmoid')(decode)

autoencoder = Model(input, decode)

autoencoder.compile(optimizer='adam',
            loss='mse',
            metrics=['accuracy'])


# Train model.
history = autoencoder.fit(train_dataset['x'], train_dataset['x'],
        epochs=100,
        batch_size=128,
        shuffle=True
        )

Epoch 1/100
754/754 [==============================] - 2s 2ms/step - loss: 0.1007 - accuracy: 0.0678
Epoch 2/100
754/754 [==============================] - 1s 1ms/step - loss: 0.0112 - accuracy: 0.0617
Epoch 3/100
754/754 [==============================] - 1s 1ms/step - loss: 0.0081 - accuracy: 0.1005
Epoch 4/100
754/754 [==============================] - 1s 1ms/step - loss: 0.0075 - accuracy: 0.1273
Epoch 5/100
754/754 [==============================] - 1s 1ms/step - loss: 0.0074 - accuracy: 0.1472
Epoch 6/100
754/754 [==============================] - 1s 1ms/step - loss: 0.0074 - accuracy: 0.1530
Epoch 7/100
754/754 [==============================] - 1s 2ms/step - loss: 0.0075 - accuracy: 0.1538
Epoch 8/100
754/754 [==============================] - 1s 2ms/step - loss: 0.0074 - accuracy: 0.1698
Epoch 9/100
754/754 [==============================] - 1s 1ms/step - loss: 0.0075 - accuracy: 0.1471
Epoch 10/100
754/754 [==============================] - 1s 2ms/step - loss: 0.0075 - accura

In [139]:
train_dataset['x_1auto'] = autoencoder.predict(train_dataset['x'])

In [140]:
from sklearn.metrics import mean_squared_error

In [141]:
mean_squared_error(train_x, autoencoder.predict(train_x))

0.00784969178497342

In [142]:
clf = IsolationForest(random_state = 42, contamination = 0.3)
clf.fit(train_dataset['x_1auto'])
pred = clf.predict(train_dataset['x_1auto'])

p1 = train_dataset['x_1auto'][pred==1]
p2 = train_dataset['x_1auto'][pred==-1]

In [143]:
print('Normal data count : {}'.format(len(p1)))
print('Abnormal data count : {}'.format(len(p2)))

Normal data count : 67548
Abnormal data count : 28911


In [144]:
# NN Autoencoder model.
input_dim = p1.shape[1]
input = Input(shape=(input_dim, ))
encode = Dense(input_dim//3*2, activation='relu',kernel_regularizer=regularizers.l2(0.01))(input)
encode = BatchNormalization()(encode)
encode = Dense(input_dim//3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)
encode = Dense(1, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)

decode = Dense(input_dim//3, activation='relu')(encode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim//3*2, activation='relu')(decode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim, activation='sigmoid')(decode)

autoencoder = Model(input, decode)

autoencoder.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

# Train model.
history = autoencoder.fit(p1, p1,
        epochs=100,
        batch_size=128,
        shuffle=True
        )

Epoch 1/100
528/528 [==============================] - 2s 2ms/step - loss: 0.7947 - accuracy: 0.0950
Epoch 2/100
528/528 [==============================] - 1s 2ms/step - loss: 0.6224 - accuracy: 0.0972
Epoch 3/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5957 - accuracy: 0.3678
Epoch 4/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5850 - accuracy: 0.4214
Epoch 5/100
528/528 [==============================] - 1s 1ms/step - loss: 0.5815 - accuracy: 0.7198
Epoch 6/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5811 - accuracy: 0.8969
Epoch 7/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5807 - accuracy: 0.9310
Epoch 8/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5809 - accuracy: 0.9311
Epoch 9/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5807 - accuracy: 0.9353
Epoch 10/100
528/528 [==============================] - 1s 2ms/step - loss: 0.5807 - accura

## syslog

In [145]:
struct_log = './data/ha2.txt_structured.csv' # The structured log file

In [146]:
df = pd.read_csv(struct_log)

In [147]:
df.head()

,LineId,date,time,ip,Content,EventId,EventTemplate
0,1,2021-01-19,10:30:48,172.30.171.22,"BENIT_Client IP1 : 114.119.158.142, user_no : ...",73aa200a,"BENIT_Client <*> : <*>, user_no : session_id ..."
1,2,2021-01-19,10:30:48,172.30.171.22,"BENIT_Client IP2 : 114.119.158.142, user_no : ...",41de2f67,"BENIT_Client <*> : <*>, user_no : null, Header..."
2,3,2021-01-19,10:30:04,121.135.187.201,"BENIT_Client IP2 : 203.225.82.146, user_no : n...",c06227d3,"BENIT_Client <*> : <*>, user_no : null, Header..."
3,4,2021-01-19,10:30:04,121.135.187.201,"BENIT_Client IP2 : 203.225.82.146, user_no : n...",6d35b111,"BENIT_Client <*> : <*>, user_no : null, Header..."
4,5,2021-01-19,10:30:04,121.135.187.201,"BENIT_Client IP1 : 203.225.82.146, user_no : n...",73aa200a,"BENIT_Client <*> : <*>, user_no : session_id ..."


In [148]:
from utils import dataloader
from utils.preprocessing import Vectorizer_sys


batch_size = 32
hidden_size = 32
num_directions = 2
topk = 5
train_ratio = 0.2
window_size = 10
epoches = 2
num_workers = 2
device = 0 

x_, window_y_ = \
dataloader.load_sys(struct_log, window='session', window_size=window_size, train_ratio=train_ratio, split_type='uniform',Time =True)

====== Input data summary ======
Loading ./data/ha2.txt_structured.csv
Slicing 769 sessions, with window 10
Slicing done, 4191 windows generated
Train: windows


In [149]:
feature_extractor = Vectorizer_sys()
train_dataset = feature_extractor.fit_transform(x_, window_y_)

In [150]:
train_dataset['x'] = train_dataset['x'] /266

In [151]:
# Create 6 layer model.
input_dim = train_dataset['x'].shape[1]
input = Input(shape=(input_dim, ))
encode = Dense(input_dim//3*2, activation='relu',kernel_regularizer=regularizers.l2(0.01))(input)
encode = BatchNormalization()(encode)
encode = Dense(input_dim//3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)
encode = Dense(1, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)

decode = Dense(input_dim//3, activation='relu')(encode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim//3*2, activation='relu')(decode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim, activation='sigmoid')(decode)

autoencoder = Model(input, decode)

autoencoder.summary()

autoencoder.compile(optimizer='adam',
            loss='mean_squared_error',
            metrics=['accuracy'])

# Train model.
history = autoencoder.fit(train_dataset['x'], train_dataset['x'],
        epochs=100,
        batch_size=128,
        shuffle=True
        )

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 10)]              0         
_________________________________________________________________
dense_122 (Dense)            (None, 6)                 66        
_________________________________________________________________
batch_normalization_51 (Batc (None, 6)                 24        
_________________________________________________________________
dense_123 (Dense)            (None, 3)                 21        
_________________________________________________________________
batch_normalization_52 (Batc (None, 3)                 12        
_________________________________________________________________
dense_124 (Dense)            (None, 1)                 4         
_________________________________________________________________
batch_normalization_53 (Batc (None, 1)                 4  

In [152]:
 train_dataset['x_1auto'] = autoencoder.predict(train_dataset['x'])

In [153]:
clf = IsolationForest(random_state = 42, contamination = 0.3)
clf.fit(train_dataset['x_1auto'])
pred = clf.predict(train_dataset['x_1auto'])

p1 = train_dataset['x_1auto'][pred==1]
p2 = train_dataset['x_1auto'][pred==-1]

In [154]:
# Autoencoder model.
input_dim = p1.shape[1]
input = Input(shape=(input_dim, ))
encode = Dense(input_dim//3*2, activation='relu',kernel_regularizer=regularizers.l2(0.01))(input)
encode = BatchNormalization()(encode)
encode = Dense(input_dim//3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)
encode = BatchNormalization()(encode)
encode = Dense(1, activation='relu',kernel_regularizer=regularizers.l2(0.01))(encode)

decode = Dense(input_dim//3, activation='relu')(encode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim//3*2, activation='relu')(decode)
decode = BatchNormalization()(decode)
decode = Dense(input_dim, activation='sigmoid')(decode)

autoencoder = Model(input, decode)

autoencoder.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

# Train model.
history = autoencoder.fit(p1, p1,
        epochs=100,
        batch_size=128,
        shuffle=True
        )

Epoch 1/100
23/23 [==============================] - 1s 1ms/step - loss: 0.8333 - accuracy: 0.4178
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 0.7924 - accuracy: 0.4466
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 0.7567 - accuracy: 0.3922
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 0.7245 - accuracy: 0.3968
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6935 - accuracy: 0.3738
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6629 - accuracy: 0.0560
Epoch 7/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6329 - accuracy: 0.1492
Epoch 8/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6026 - accuracy: 0.1488
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5722 - accuracy: 0.1415
Epoch 10/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5423 - accuracy: 0.1437
Epoch 11/

In [162]:
max(history.history['accuracy'])

0.9270868897438049